# Experimenting in Using Time Stamps

Build a model that takes token_ids as input. The first model layer embeds the tokens_inds.

In [1]:
import os
import numpy as np
import tensorflow as tf
from dataProcessing import load_file, encodeData, insert_target, processingScriber00
from transformers import AutoTokenizer, TFCamembertForMaskedLM
from datetime import datetime
import sys

In [2]:
### instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained("jplu/tf-camembert-base", do_lower_case=True)

### punctuation encoder
punctuation_enc = {
    'O': 0,
    'PERIOD': 1,
}

### Set Vocabulary Size and Hidden Dimension (BERT)
vocabSize = 32005
hiddenDimension = 768

### hyper-parameters
sequenceSize = 32
batchSize = 32
learningRate = 1e-5
trainLayerIndex = 0
numEpo = 3

listHyper0 = ['sequenceSize', 'batchSize', 'learningRate', 'trainLayerIndex', 'numEpo']
listHyper1 = [str(sequenceSize), str(batchSize), str(learningRate), str(trainLayerIndex), str(numEpo)]
time = datetime.now().strftime("%Y%m%d_%H%M%S")
save_path = 'ModelsExpScriber/{}/'.format(time)
os.mkdir(save_path)

In [6]:
### Get Training Dataset

print('\nProcessing Training Data ... ')

# name of dataset with sentences
data_name = "Scriber"

# file name
trainDataName = 'Data' + data_name + '/' + 'raw.processed.Train_01.txt'

# from sentences to list of words+punctuation
data_train = load_file(processingScriber00(trainDataName))

# encode data and insert target
X_train_, y_train_ = encodeData(data_train, tokenizer, punctuation_enc)
X_train = insert_target(X_train_, sequenceSize)
y_train = np.asarray(y_train_)

# get only a fraction of data 
n = 1821184
X_train = X_train[0:n]
y_train = y_train[0:n]

# build the datasets
trainDataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(buffer_size=500000).batch(batchSize)

print("\nTraining Dataset Tensor Shape = ", X_train.shape)


Processing Training Data ... 

Training Dataset Tensor Shape =  (1821184, 32)


In [7]:
# (1821205-21)
1821184 /32

56912.0

In [ ]:
### Build a Test Input Tensor 

testInputTensor, label = next(iter(trainDataset))
# print(testInputTensor.shape)
# print(testInputTensor)
testInputTensor = tf.one_hot(testInputTensor, vocabSize)
# print("\n", testInputTensor.shape)
# print(testInputTensor[0][0, 50])

In [ ]:
### Get word_embeddings Weights From a Pre-Trained Model

# Build The Model
bert_input = tf.keras.Input(shape=(sequenceSize), dtype='int32')
x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(bert_input)[0]
x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
dense_out = tf.keras.layers.Dense(len(punctuation_enc))(x)
modelPreTrained = tf.keras.Model(bert_input, dense_out)

# Extract word_embeddings Weights
weightsTensor = modelPreTrained.trainable_variables[194][0:]

# print some weights
print(modelPreTrained.trainable_variables[194][0][0:5])

In [ ]:
### Build a Linear Layer to Embed the tokens_ids

linearProj = tf.keras.layers.Dense(768, input_shape=(32005,), use_bias=False)
tensor = tf.zeros(shape=[1, 32005])
linearProj(tensor)  # execute the layer
linearProj.trainable_variables[0].assign(weightsTensor)  # load the weights

# print some weights
print(linearProj.trainable_variables[0][0][0:5])

In [ ]:
### Build a BERT Layer and Execute It

bertLayer = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")
randTensor = tf.random.uniform(shape=[batchSize, sequenceSize, hiddenDimension])
_ = bertLayer(None, inputs_embeds=randTensor)

In [ ]:
### Build the Experimental Model

inp = tf.keras.Input(shape=(sequenceSize, vocabSize), batch_size=batchSize, dtype='float32')
x = linearProj(inp)
x = bertLayer(None, inputs_embeds=x)[0]
x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
out = tf.keras.layers.Dense(len(punctuation_enc), activation='softmax')(x)

model = tf.keras.Model(inp, out)

In [ ]:
### Runt the Model on One Input

_ = model(testInputTensor)

In [ ]:
### Define Loss

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y):
    y_ = model(x)
    return loss_object(y_true=y, y_pred=y_)

### Function to Calculate Gradients

def grad(model, inputs, targets, trainLayInd):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return loss_value, tape.gradient(loss_value, model.trainable_variables[trainLayInd:])

### Define the Optimizer

optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate)

### Training Loop

In [ ]:
print("\nExperiment Folder: ", time)
print("\nHyperparameters:")
print('vocabSize = ', vocabSize)
print('sequenceSize = ', sequenceSize)
print('batchSize = ', batchSize)
print('leaRat = ', learningRate)
print('Train Layer Index = ', trainLayerIndex)
print('numEpo = ', numEpo)

epoch_loss_avg = tf.keras.metrics.Mean()
epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

train_loss_results = []
train_accuracy_results = []

checkpoint_path = save_path + "cp-{epoch:03d}.ckpt"

print("\nTraining the Model ... ")
for epoch in range(1, numEpo+1):

    # training loop
    for x, y in trainDataset:
        x = tf.one_hot(x, vocabSize)
        # optimize the model
        loss_value, grads = grad(model, x, y, trainLayerIndex)
        optimizer.apply_gradients(zip(grads, model.trainable_variables[trainLayerIndex:]))

        # track progress
        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(y, model(x))

    # end epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    print("\nEpoch {:03d}: (Training)   Loss: {:.3f}, Accuracy: {:.3%}".format(epoch, epoch_loss_avg.result(), epoch_accuracy.result()))

    # # save model if new min for train loss is found
    tmpTrain = epoch_loss_avg.result().numpy()
    model.save_weights(checkpoint_path.format(epoch=epoch))

    epoch_loss_avg.reset_states()
    epoch_accuracy.reset_states()

### Output Training Details on Log File

In [ ]:
nameLogFile = 'log.txt'
logFile = open(save_path + nameLogFile, "w")

# write name of model
logFile.write("\n" + time + "\n\n")

# write hyper parameters
for i in range(len(listHyper0)):
    logFile.write(listHyper0[i] + ":  " + listHyper1[i] + "\n")

# write training details
logFile.write('\nTRAINING')
trainLossArr = np.asarray(train_loss_results)
trainAccArr = np.asarray(train_accuracy_results)
for i in range(numEpo):
    epoch = i+1
    logFile.write("\nEpoch {:03d}:   Loss: {:7.4f},   Accuracy: {:7.4%}".format(epoch, trainLossArr[i], trainAccArr[i]))

### Evaluate the models, Write Results on the logFile

In [ ]:
### Get the Test Dataset

# name of dataset
dataName = "Scriber"

# file name
testDataName = 'Data' + data_name + '/' + 'raw.processed.Test_01.txt'

# from sentences to list of words+punctuation
data = load_file(processingScriber00(testDataName))

# Encode data and insert target.
X_, y_ = encodeData(data, tokenizer, punctuation_enc)
X = insert_target(X_, sequenceSize)
y = np.asarray(y_)

# # get only a fraction of data 
# n = 64
# X = X[0:n]
# y = y[0:n]

# one hot encode the labels
y = tf.one_hot(y, len(punctuation_enc), dtype='int64').numpy()

# build the datasets
dataset = tf.data.Dataset.from_tensor_slices((X, y)).batch(batchSize)

print("\nTest Dataset Tensor Shape = ", X.shape)

In [ ]:
### Build and Compile the Model

inp = tf.keras.Input(shape=(sequenceSize, vocabSize), batch_size=batchSize, dtype='float32')
x = linearProj(inp)
x = bertLayer(None, inputs_embeds=x)[0]
x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
out = tf.keras.layers.Dense(len(punctuation_enc), activation='softmax')(x)

model = tf.keras.Model(inp, out)

model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[tf.keras.metrics.Recall(class_id=0, name='Rec_0'),
                       tf.keras.metrics.Precision(class_id=0, name='Prec_0'),
                       tf.keras.metrics.Recall(class_id=1, name='Rec_1'),
                       tf.keras.metrics.Precision(class_id=1, name='Prec_1'),
                      ])

In [ ]:
### Get List of the Models in the Output Folder

modelsLst = []
for r, d, f in os.walk(save_path):
    for file in sorted(f):
        if ".index" in file:
            modelsLst.append(file[:-6])

In [ ]:
# compute f1 score
def compF1(rec, pre):
    if pre + rec == .0:
        return .0
    else:
        return 2 * (pre*rec) / (pre+rec)

In [ ]:
# ### Evaluate the Models

# print("\nEvaluate Models")

# print("\nTest Set Tensor Shape = ", X.shape)

# logFile.write('\n\nEVALUATION\n')
# for i in range(len(modelsLst)):
#     checkpointPath = save_path + modelsLst[i]
#     print(checkpointPath)

#     # load weights
#     model.load_weights(checkpointPath)

#     # evaluate
#     evaluation = model.evaluate(dataset)
    
#     f1_0 = compF1(evaluation[1],evaluation[2])
#     f1_1 = compF1(evaluation[3],evaluation[4])
#     print("F1_0 = {:10.7f} - F1_1 = {:10.7f}".format(f1_0, f1_1))
    
#     # write details on log files
#     logFile.write(modelsLst[i])
#     logFile.write(" - Loss = {:7.4f} - Rec_0 = {:6.4f} - Pre_0 = {:6.4f} - F1_0 = {:10.7f} - Rec_1 = {:6.4f} - Pre_1 = {:6.4f} - F1_1 = {:10.7f}\n".format(evaluation[0], evaluation[1], evaluation[2], f1_0, evaluation[3], evaluation[4], f1_1))

# logFile.close()